In [1]:
import pandas as pd
from datetime import datetime
import os

In [2]:
APPLICATION_BASE_PATH = "D:/PhD/Model_Building/Stage_4_Building_Vulnerability_Dataset"
round_path = APPLICATION_BASE_PATH+"/8_Round_Qark"
data_csv = round_path+"/8_Round_App_Qark_Analysis_CWE_mapped.csv"
datasets_path =round_path+"/Scanned_App_Unprocessed_Dataset_Parts"

TYPE = "Scanned_App"
#TYPE = "Scanned_Files"

In [3]:
df_data = pd.read_csv(data_csv)

In [4]:
df_data.head()

,Unnamed: 0,app_name,category,severity,description,qark_name,line_number,file_object,apk_exploit_dict,CWE_ID,CWE_Desc
0,0,yalp-store-fork,file,WARNING,Logs are detected. This may allow potential le...,Logging found,[73_ 7],I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-532,Insertion of Sensitive Information into Log File
1,1,yalp-store-fork,file,WARNING,Logs are detected. This may allow potential le...,Logging found,[63_ 13],I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-532,Insertion of Sensitive Information into Log File
2,2,yalp-store-fork,file,WARNING,Logs are detected. This may allow potential le...,Logging found,[39_ 13],I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-532,Insertion of Sensitive Information into Log File
3,3,yalp-store-fork,manifest,WARNING,Be careful with use of Check permission functi...,Potientially vulnerable check permission funct...,None,I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-732,Incorrect Permission Assignment for Critical R...
4,4,yalp-store-fork,manifest,WARNING,Be careful with use of Check permission functi...,Potientially vulnerable check permission funct...,None,I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-732,Incorrect Permission Assignment for Critical R...


In [5]:
df_data.set_index('app_name')
df_data.head()

,Unnamed: 0,app_name,category,severity,description,qark_name,line_number,file_object,apk_exploit_dict,CWE_ID,CWE_Desc
0,0,yalp-store-fork,file,WARNING,Logs are detected. This may allow potential le...,Logging found,[73_ 7],I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-532,Insertion of Sensitive Information into Log File
1,1,yalp-store-fork,file,WARNING,Logs are detected. This may allow potential le...,Logging found,[63_ 13],I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-532,Insertion of Sensitive Information into Log File
2,2,yalp-store-fork,file,WARNING,Logs are detected. This may allow potential le...,Logging found,[39_ 13],I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-532,Insertion of Sensitive Information into Log File
3,3,yalp-store-fork,manifest,WARNING,Be careful with use of Check permission functi...,Potientially vulnerable check permission funct...,None,I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-732,Incorrect Permission Assignment for Critical R...
4,4,yalp-store-fork,manifest,WARNING,Be careful with use of Check permission functi...,Potientially vulnerable check permission funct...,None,I:\PhD\Model_Building\APKs_and_GitRepos\8_Roun...,None,CWE-732,Incorrect Permission Assignment for Critical R...


In [6]:
df_data.shape

(16348, 11)

In [7]:
def read_source_file(file_path):
    file_df = pd.read_csv(file_path, names=['code_line'], sep="\t", skip_blank_lines=False).fillna("")
    return file_df

In [8]:
def generate_dataset(start, end):
    vulberability_dataset = []  
    for i in range(start, end):
        print("Processing line :",i)
        row = df_data.iloc[i]
        try:
            app_name = row.app_name
            category = row.category
            severity = row.severity
            qark_name = row.qark_name    
            line_number = int(str(row.line_number).split("_")[0][1:])
            file_object = row.file_object
            apk_exploit_dict = row.apk_exploit_dict
            CWE_ID = row.CWE_ID
            CWE_Desc = row.CWE_Desc 
            try:

                source_file = read_source_file(file_object)
                file_type = str(file_object).split(".")[1].lower()
                if(file_type not in ['java', 'kt', 'xml', 'txt']):
                    continue

                for j, file_row in source_file.iterrows():
                    code_line = file_row.code_line.strip()
                    if(j == line_number-1):
                        vulnerability_status = 1

                        csv_data_row = [app_name, category, severity, qark_name, code_line, CWE_ID, CWE_Desc,vulnerability_status]
                    else:
                        vulnerability_status = 0
                        csv_data_row = [app_name, "", "", "", code_line, "", "",vulnerability_status]       


                    vulberability_dataset.append(csv_data_row)
            except Exception as e:
                print(e)
                continue
        except Exception as ef:
            print(ef)
            continue
        print("Processed line :",i)


    header = ['app_name', 'category', 'severity', 'qark_name', 'Code', 'CWE_ID', 'CWE_Desc', 'Vulnerability_status']
    vulberability_dataframe = pd.DataFrame(vulberability_dataset, columns = header)

    vulberability_dataframe.drop_duplicates(subset=['Code', 'CWE_ID'], inplace=True)

    vulberability_dataframe.to_csv(datasets_path+"/"+TYPE+'_Unprocessed_Dataset_'+str(start)+'_'+str(end)+'.csv', sep=',', encoding='utf-8', index=False)


In [9]:
def execute_generation_process():
    dataset_length = len(df_data)
    increment = 2500

    for i in range(0,dataset_length, increment):
        start = i
        end = i+increment
        if(end>dataset_length):
            end= dataset_length

        print(str(datetime.now())+": Dataset generating : "+ str(start)+"_"+ str(end))
        generate_dataset(start,end)

In [11]:
execute_generation_process()

2022-04-01 13:01:01.566683: Dataset generating : 0_2500
Processing line : 0
Processed line : 0
Processing line : 1
Processed line : 1
Processing line : 2
Processed line : 2
Processing line : 3
invalid literal for int() with base 10: 'one'
Processing line : 4
invalid literal for int() with base 10: 'one'
Processing line : 5
Processed line : 5
Processing line : 6
Processed line : 6
Processing line : 7
Processed line : 7
Processing line : 8
Processed line : 8
Processing line : 9
Processed line : 9
Processing line : 10
Processed line : 10
Processing line : 11
Processed line : 11
Processing line : 12
Processed line : 12
Processing line : 13
Processed line : 13
Processing line : 14
Processed line : 14
Processing line : 15
Processed line : 15
Processing line : 16
Processed line : 16
Processing line : 17
Processed line : 17
Processing line : 18
Processed line : 18
Processing line : 19
Processed line : 19
Processing line : 20
Processed line : 20
Processing line : 21
Processed line : 21
Processi

Processing line : 206
Processed line : 206
Processing line : 207
Processed line : 207
Processing line : 208
Processed line : 208
Processing line : 209
Processed line : 209
Processing line : 210
Processed line : 210
Processing line : 211
Processed line : 211
Processing line : 212
Processed line : 212
Processing line : 213
Processed line : 213
Processing line : 214
Processed line : 214
Processing line : 215
Processed line : 215
Processing line : 216
Processed line : 216
Processing line : 217
Processed line : 217
Processing line : 218
Processed line : 218
Processing line : 219
Processed line : 219
Processing line : 220
Processed line : 220
Processing line : 221
Processed line : 221
Processing line : 222
Processed line : 222
Processing line : 223
Processed line : 223
Processing line : 224
Processed line : 224
Processing line : 225
Processed line : 225
Processing line : 226
invalid literal for int() with base 10: 'one'
Processing line : 227
invalid literal for int() with base 10: 'one'
Proc

Processed line : 393
Processing line : 394
Processed line : 394
Processing line : 395
Processed line : 395
Processing line : 396
Processed line : 396
Processing line : 397
invalid literal for int() with base 10: 'one'
Processing line : 398
Processed line : 398
Processing line : 399
Processing line : 400
Processed line : 400
Processing line : 401
Processed line : 401
Processing line : 402
Processed line : 402
Processing line : 403
Processed line : 403
Processing line : 404
Processed line : 404
Processing line : 405
Processed line : 405
Processing line : 406
Processed line : 406
Processing line : 407
Processed line : 407
Processing line : 408
Processed line : 408
Processing line : 409
Processed line : 409
Processing line : 410
Processing line : 411
Processed line : 411
Processing line : 412
Processed line : 412
Processing line : 413
invalid literal for int() with base 10: 'one'
Processing line : 414
Processed line : 414
Processing line : 415
Processed line : 415
Processing line : 416
inv

Processed line : 584
Processing line : 585
invalid literal for int() with base 10: 'one'
Processing line : 586
Processed line : 586
Processing line : 587
Processed line : 587
Processing line : 588
Processed line : 588
Processing line : 589
Processed line : 589
Processing line : 590
Processed line : 590
Processing line : 591
Processed line : 591
Processing line : 592
Processed line : 592
Processing line : 593
Processed line : 593
Processing line : 594
Processed line : 594
Processing line : 595
Processed line : 595
Processing line : 596
Processed line : 596
Processing line : 597
Processed line : 597
Processing line : 598
Processed line : 598
Processing line : 599
Processed line : 599
Processing line : 600
Processed line : 600
Processing line : 601
Processed line : 601
Processing line : 602
Processed line : 602
Processing line : 603
Processed line : 603
Processing line : 604
Processed line : 604
Processing line : 605
Processed line : 605
Processing line : 606
Processed line : 606
Processi

Processed line : 775
Processing line : 776
Processed line : 776
Processing line : 777
Processed line : 777
Processing line : 778
Processed line : 778
Processing line : 779
Processed line : 779
Processing line : 780
Processed line : 780
Processing line : 781
Processed line : 781
Processing line : 782
Processed line : 782
Processing line : 783
Processed line : 783
Processing line : 784
Processed line : 784
Processing line : 785
Processed line : 785
Processing line : 786
Processed line : 786
Processing line : 787
Processed line : 787
Processing line : 788
Processed line : 788
Processing line : 789
Processed line : 789
Processing line : 790
Processed line : 790
Processing line : 791
Processed line : 791
Processing line : 792
Processed line : 792
Processing line : 793
Processed line : 793
Processing line : 794
Processed line : 794
Processing line : 795
Processed line : 795
Processing line : 796
Processed line : 796
Processing line : 797
Processed line : 797
Processing line : 798
Processed l

...

Processed line : 15615
Processing line : 15616
invalid literal for int() with base 10: 'one'
Processing line : 15617
Processed line : 15617
Processing line : 15618
Processed line : 15618
Processing line : 15619
Processed line : 15619
Processing line : 15620
Processed line : 15620
Processing line : 15621
Processed line : 15621
Processing line : 15622
Processed line : 15622
Processing line : 15623
Processed line : 15623
Processing line : 15624
Processed line : 15624
Processing line : 15625
Processed line : 15625
Processing line : 15626
Processed line : 15626
Processing line : 15627
Processed line : 15627
Processing line : 15628
Processed line : 15628
Processing line : 15629
Processed line : 15629
Processing line : 15630
Processed line : 15630
Processing line : 15631
Processed line : 15631
Processing line : 15632
Processed line : 15632
Processing line : 15633
Processed line : 15633
Processing line : 15634
Processed line : 15634
Processing line : 15635
Processed line : 15635
Processing lin

Processed line : 15790
Processing line : 15791
Processed line : 15791
Processing line : 15792
Processed line : 15792
Processing line : 15793
Processed line : 15793
Processing line : 15794
Processed line : 15794
Processing line : 15795
Processed line : 15795
Processing line : 15796
invalid literal for int() with base 10: 'one'
Processing line : 15797
Processed line : 15797
Processing line : 15798
Processed line : 15798
Processing line : 15799
Processing line : 15800
Processed line : 15800
Processing line : 15801
Processed line : 15801
Processing line : 15802
Processed line : 15802
Processing line : 15803
Processed line : 15803
Processing line : 15804
Processed line : 15804
Processing line : 15805
Processed line : 15805
Processing line : 15806
Processed line : 15806
Processing line : 15807
Processed line : 15807
Processing line : 15808
Processed line : 15808
Processing line : 15809
Processed line : 15809
Processing line : 15810
Processed line : 15810
Processing line : 15811
Processed lin

Processed line : 15966
Processing line : 15967
Processed line : 15967
Processing line : 15968
Processed line : 15968
Processing line : 15969
Processed line : 15969
Processing line : 15970
Processed line : 15970
Processing line : 15971
Processed line : 15971
Processing line : 15972
Processed line : 15972
Processing line : 15973
Processed line : 15973
Processing line : 15974
Processed line : 15974
Processing line : 15975
Processed line : 15975
Processing line : 15976
Processed line : 15976
Processing line : 15977
Processed line : 15977
Processing line : 15978
Processed line : 15978
Processing line : 15979
Processed line : 15979
Processing line : 15980
Processed line : 15980
Processing line : 15981
Processed line : 15981
Processing line : 15982
Processed line : 15982
Processing line : 15983
Processed line : 15983
Processing line : 15984
Processed line : 15984
Processing line : 15985
Processed line : 15985
Processing line : 15986
Processed line : 15986
Processing line : 15987
Processed lin

Processed line : 16137
Processing line : 16138
Processed line : 16138
Processing line : 16139
Processed line : 16139
Processing line : 16140
Processed line : 16140
Processing line : 16141
Processed line : 16141
Processing line : 16142
Processed line : 16142
Processing line : 16143
Processed line : 16143
Processing line : 16144
Processed line : 16144
Processing line : 16145
Processed line : 16145
Processing line : 16146
Processed line : 16146
Processing line : 16147
Processed line : 16147
Processing line : 16148
Processed line : 16148
Processing line : 16149
Processed line : 16149
Processing line : 16150
Processed line : 16150
Processing line : 16151
Processed line : 16151
Processing line : 16152
Processed line : 16152
Processing line : 16153
Processed line : 16153
Processing line : 16154
Processed line : 16154
Processing line : 16155
Processed line : 16155
Processing line : 16156
Processed line : 16156
Processing line : 16157
Processed line : 16157
Processing line : 16158
Processed lin

Processed line : 16304
Processing line : 16305
Processed line : 16305
Processing line : 16306
Processed line : 16306
Processing line : 16307
Processed line : 16307
Processing line : 16308
Processed line : 16308
Processing line : 16309
Processed line : 16309
Processing line : 16310
Processed line : 16310
Processing line : 16311
Processed line : 16311
Processing line : 16312
Processed line : 16312
Processing line : 16313
Processed line : 16313
Processing line : 16314
Processed line : 16314
Processing line : 16315
Processed line : 16315
Processing line : 16316
Processed line : 16316
Processing line : 16317
Processed line : 16317
Processing line : 16318
Processed line : 16318
Processing line : 16319
Processed line : 16319
Processing line : 16320
Processed line : 16320
Processing line : 16321
Processed line : 16321
Processing line : 16322
Processed line : 16322
Processing line : 16323
Processed line : 16323
Processing line : 16324
Processed line : 16324
Processing line : 16325
Processed lin

In [12]:
def combine_dfs():
    df_all =  pd.DataFrame()
    files = os.listdir(datasets_path)
    
    for file in files:
        data_csv = datasets_path+"/"+file
        df_current = pd.read_csv(data_csv).fillna("")        
        df_all = pd.concat([df_all,df_current],ignore_index=True)
    
    
    df_all.drop_duplicates(subset=['Code', 'CWE_ID'], inplace=True)
    
    df_all.to_csv(round_path+"/Scanned_Files_Unprocessed_Dataset.csv")    
    print (df_all.shape)

In [13]:
combine_dfs()

(291958, 8)
